In [1]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import os

In [4]:
# collect data and transfer into numpy array using opencv

In [5]:
import cv2 as cv

In [12]:
path='data'

In [13]:
labels=['without_mask','with_mask']

In [15]:
x=[]
y=[]

In [19]:
for names in os.listdir(path):
    indx=labels.index(names)
    for image in os.listdir(os.path.join(path,names)):
        img_path=os.path.join(path,names,image)
        img=cv.imread(img_path)
        resized=cv.resize(img,(128,128))
        x.append(resized)
        y.append(indx)
        
    

In [97]:
# checking if both lists have the same lenght
len(x),len(y)

TypeError: object of type 'numpy.float64' has no len()

In [93]:
# converting to numpy array
x=np.array(x)
y=np.array(y)

In [94]:
# scaling the x values for better accuracy
x=x/255

In [95]:
x.max()

0.9019607843137255

In [96]:
x.shape,y.shape

((), ())

In [26]:
# seeing the images at random
import random

In [98]:
index=random.randint(0,2500)
plt.imshow(x[index])
plt.xlabel(y[index])

IndexError: invalid index to scalar variable.

In [36]:
# divide the data in training data and testing data

In [37]:
from sklearn.model_selection import train_test_split

In [92]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.15)

TypeError: Singleton array 230 cannot be considered a valid collection.

In [39]:
x_train.shape,y_train.shape

((2464, 128, 128, 3), (2464,))

In [43]:
x_train.max(),y_train.shape

(1.0, (2464,))

# creating the cnn model

In [41]:
model=keras.Sequential()
# cnn
model.add(keras.layers.Conv2D(45,kernel_size=(3,3),activation='relu',input_shape=(128,128,3)))
model.add(keras.layers.MaxPool2D())

model.add(keras.layers.Conv2D(30,kernel_size=(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D())

# flattening 
model.add(keras.layers.Flatten())
# deep nn
model.add(keras.layers.Dense(64,activation='relu'))
model.add(keras.layers.Dense(32,activation='relu'))
model.add(keras.layers.Dense(1,activation='sigmoid'))

In [42]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# training the model

In [44]:
model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
77/77 [==============================] - 56s 686ms/step - loss: 0.4268 - accuracy: 0.7942 - val_loss: 0.2518 - val_accuracy: 0.8782
Epoch 2/5
77/77 [==============================] - 61s 799ms/step - loss: 0.2132 - accuracy: 0.9144 - val_loss: 0.2074 - val_accuracy: 0.9126
Epoch 3/5
77/77 [==============================] - 64s 825ms/step - loss: 0.1604 - accuracy: 0.9444 - val_loss: 0.2289 - val_accuracy: 0.9149
Epoch 4/5
77/77 [==============================] - 58s 748ms/step - loss: 0.1079 - accuracy: 0.9639 - val_loss: 0.2260 - val_accuracy: 0.9103
Epoch 5/5
77/77 [==============================] - 52s 680ms/step - loss: 0.0877 - accuracy: 0.9748 - val_loss: 0.1725 - val_accuracy: 0.9402


In [46]:
model.fit(x_train,y_train,epochs=1,validation_data=(x_test,y_test))

77/77 [==============================] - 50s 646ms/step - loss: 0.0308 - accuracy: 0.9903 - val_loss: 0.2513 - val_accuracy: 0.9310


In [47]:
model.evaluate(x_test,y_test)

14/14 [==============================] - 2s 159ms/step - loss: 0.2513 - accuracy: 0.9310


[0.25133123993873596, 0.931034505367279]

In [51]:
def get_predictions(y):
    l=[]
    for i in y:
        i=i[0]
        if i>=0.5:
            l.append('mask')
        else:
            l.append('no mask')
    return l[0]

In [49]:
y_p=model.predict(x_test)

14/14 [==============================] - 3s 151ms/step


In [55]:
get_predictions(y_p)[:10]

['mask',
 'no mask',
 'no mask',
 'no mask',
 'no mask',
 'no mask',
 'no mask',
 'no mask',
 'no mask',
 'mask']

In [56]:
y_test[:10]

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 1])

In [73]:
def give_result(img):
    res_img=cv.resize(img,(128,128))
    res_img=res_img/255
    y_p=model.predict(np.expand_dims(res_img,0))
    print(y_p)
    result=get_predictions(y_p)
    return result,y_p[0][0]

In [85]:
img=cv.imread('mask5.jpg')


In [86]:
give_result(img)

1/1 [==============================] - 0s 42ms/step
[[0.8536359]]


(['mask'], 0.8536359)

In [60]:
img=cv.imread('nomask.jpg')

In [61]:
give_result(img)

1/1 [==============================] - 0s 43ms/step


['mask']

In [100]:
image=cv.imread('no.jpeg')
give_result(image)

1/1 [==============================] - 0s 31ms/step
[[0.00024585]]


(['no mask'], 0.00024584532)

# realtime detection

In [67]:
face_cascade=cv.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

In [102]:
cap=cv.VideoCapture(0)
while True:
    try:
        _,frame=cap.read()
#         gray=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
#         faces=face_cascade.detectMultiScale(gray,1.1,4)
#         x,y,w,h=faces[0]
        label,pro=give_result(frame)
        cv.putText(frame,f"{label} {round(pro*100,2)}",(20,20),cv.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,255),1,cv.LINE_AA)
#         cv.rectangle(frame,(x,y),(x+h,y+h),(0,255,0),1,cv.LINE_AA)
    except:
        label,pro=give_result(frame)
        print(label)
        cv.putText(frame,f"{label} {round(pro*100,2)}",(20,20),cv.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,255),1,cv.LINE_AA)
    if cv.waitKey(1)==27:
        break
    
    cv.imshow('frame',frame)
    
    
cap.release()
cv.destroyAllWindows()

1/1 [==============================] - 0s 53ms/step
[[0.06251245]]
1/1 [==============================] - 0s 24ms/step
[[0.10325986]]
1/1 [==============================] - 0s 32ms/step
[[0.10701481]]
1/1 [==============================] - 0s 39ms/step
[[0.05540916]]
1/1 [==============================] - 0s 36ms/step
[[0.01827127]]
1/1 [==============================] - 0s 32ms/step
[[0.01805785]]
1/1 [==============================] - 0s 35ms/step
[[0.01926839]]
1/1 [==============================] - 0s 30ms/step
[[0.01926839]]
1/1 [==============================] - 0s 24ms/step
[[0.02567372]]
1/1 [==============================] - 0s 40ms/step
[[0.03023271]]
1/1 [==============================] - 0s 32ms/step
[[0.03086417]]
1/1 [==============================] - 0s 36ms/step
[[0.03068035]]
1/1 [==============================] - 0s 38ms/step
[[0.03503921]]
1/1 [==============================] - 0s 35ms/step
[[0.03429243]]
1/1 [==============================] - 0s 32ms/step
[[0.034292

1/1 [==============================] - 0s 24ms/step
[[0.05540917]]
1/1 [==============================] - 0s 32ms/step
[[0.05629791]]
1/1 [==============================] - 0s 32ms/step
[[0.05264803]]
1/1 [==============================] - 0s 32ms/step
[[0.05616409]]
1/1 [==============================] - 0s 24ms/step
[[0.06231549]]
1/1 [==============================] - 0s 32ms/step
[[0.06339835]]
1/1 [==============================] - 0s 37ms/step
[[0.06339835]]
1/1 [==============================] - 0s 23ms/step
[[0.06349446]]
1/1 [==============================] - 0s 24ms/step
[[0.0641719]]
1/1 [==============================] - 0s 39ms/step
[[0.0625257]]
1/1 [==============================] - 0s 24ms/step
[[0.05902204]]
1/1 [==============================] - 0s 32ms/step
[[0.05590972]]
1/1 [==============================] - 0s 32ms/step
[[0.05590972]]
1/1 [==============================] - 0s 32ms/step
[[0.06857588]]
1/1 [==============================] - 0s 32ms/step
[[0.08378224

In [89]:
model.save('last.h5')